# KerasによるDeep Neural Network実装(MNIST)
PythonのDeep Learningライブラリ Kerasを用いてDeep Neural Networkの実装をします．

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import matplotlib.pyplot as plt
import seaborn as sns
import h5py

from matplotlib import rcParams
rcParams["figure.figsize"] = [15, 5]
rcParams['xtick.labelsize'] = 15
rcParams['ytick.labelsize'] = 15
rcParams['axes.labelsize'] = 15
rcParams['axes.titlesize'] = 18
rcParams['axes.grid'] = True
rcParams['legend.fontsize'] = 15

%matplotlib inline

- `keras.layers` からは，ディープラーニングで用いることのできる様々なレイヤーをimportできます．特に基本的なレイヤーは，`keras.layers.core`にまとめられています．詳細は公式リファレンスを参照してください．

Core Layers: https://keras.io/ja/layers/core/

## 1. データの準備

- MNISTをはじめ，いくつかのデータセットはKerasからimportすることができます．


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

- ほとんどの場合，データセットの前処理が必要です．ロードしたデータのshapeを確認し，学習に適した形式に変換します．

In [0]:
print("x_train:",x_train.shape)
print("y_train:",y_train.shape)
print("x_test:",x_test.shape)
print("y_test:",y_test.shape)

In [0]:
# 28x28 の行列を 784x1 のベクトルに変換してください 

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

- 画素値の正規化を行います．通常$0-255$の値をとる画素値を，$0-1$の範囲に変換します．

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

- クラス分類タスクでは，正解ラベルをOne-hot Label形式で表現します．kerasでは，`keras.utils.to_categorical`関数で変換できます．

In [0]:
# 変換前

y_train[:10]

In [0]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [0]:
# 変換後

y_train[:10]

- 最後に，データセットの次元を確認します．

In [0]:
print("x_train:",x_train.shape)
print("y_train:",y_train.shape)
print("x_test:",x_test.shape)
print("y_test:",y_test.shape)

## 2. モデル構築

- SequentialモデルでDNNを構築します．`model = Sequential()`でSequentialモデルを宣言します．
- 一つのレイヤーは`Dense`レイヤーを追加することで実装できます．`Dense`では，ユニット数と活性化を指定できます．
- 一番初めの層だけは`input_shape`引数が必要です．入力の次元を指定します．

In [0]:
# モデルの構築
model = Sequential()

# 最初の隠れ層(入力784 -> 出力512)
model.add(Dense(512, activation="relu",input_shape=(784,)))
model.add(Dropout(0.7))

# 2つ目の隠れ層(入力512 -> 出力256)
## 隠れ層は以下の2行をコピペすることで追加できます
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.7))

# 出力層(入力256 -> 出力10)
model.add(Dense(10, activation='softmax'))

- `model.summary()`関数で，構築したモデルの概要が確認できます．

In [0]:
model.summary()

- 構築したモデルをコンパイルします．コンパイルには，損失関数，最適化手法と評価関数が設定できます．
- 損失関数は次のものが使用できます: https://keras.io/ja/losses/
- 最適化手法は次のものが使用できます: https://keras.io/ja/optimizers/
  - `optimizer="XXX"` の引数で指定する場合、挙げられている手法を小文字にして入力してください。
  - (ex) Adagradを使用したい場合 `optimizer="adagrad"`
- 評価関数は，未指定の場合はlossが採用されます．その他いくつか使用可能なものがありますが，一般にはaccuracyを用いると良いでしょう．

In [0]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=['accuracy'])

- バッチサイズ，エポック数を指定し，`model.fit`関数で学習を開始できます．
- `model.fit`をもう一度実行すると，前回の学習状態からスタートします．リセットしたい場合は，`model.reset_states()`を再実行してから`model.fit`を実行します．

In [0]:
# バッチサイズ
batch_size = 2000

# エポック数
epochs = 2

In [0]:
# フィッティング(学習)
history = model.fit(x_train, y_train,
                                   batch_size=batch_size,
                                   epochs=epochs,
                                   validation_data=(x_test, y_test),
                                   verbose=1)

- `model.evaluate`関数によって現在のモデルの精度が確認できます．

In [0]:
# テストスコアの計算・表示
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 3. モデルの評価

## 3-1. 学習過程の可視化

- エポックごとのLossとAccuracyを可視化します．
- 過学習やUnder Fittingの確認をすることができます．

In [0]:
loss = history.history['loss']
val_loss = history.history['val_loss']

acc = history.history['acc']
val_acc = history.history['val_acc']

plt.figure(figsize=(15,5))
plt.rcParams["font.size"] = "20"
plt.title('Loss')
epochs = len(loss)
plt.plot(range(epochs), loss, marker='.', label='loss')
plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
plt.legend(loc='best')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

plt.figure(figsize=(15,5))
plt.title('Accuracy')
plt.plot(range(epochs), acc, marker='.', label='acc')
plt.plot(range(epochs), val_acc, marker='.', label='val_acc')
plt.legend(loc='best')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

## 3-2. Confusion Matrix

- `sklearn`の`confusion_matrix`を用いると，簡単にconfusion matrixを求めることができます．`sklearn`は最も有名なPythonの機械学習ライブラリの一つです．
- `model.predict_classes`関数を用いると，現在のモデルを用いたクラス分類結果を，クラスのインデックスで得ることができます．

In [0]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

y_pred = model.predict_classes(x_test, batch_size=32, verbose=2)
y_gt = np.apply_along_axis(np.argmax,1,y_test)

conf_matrix = confusion_matrix(y_gt,y_pred)

- 今回は，クラス分類結果数のマトリックスと，正解クラス別の予測分類結果の割合のマトリックスを表示してみます．

In [0]:
conf_cnt_df = pd.DataFrame(conf_matrix)
conf_rate_df = pd.DataFrame(conf_matrix/np.sum(conf_matrix, axis=1))

plt.figure(figsize = (8,8))
sns.heatmap(conf_cnt_df, annot=True, cmap='RdYlGn', linewidths=0.1, vmin=0)
plt.xlabel("Predict", fontsize=20); plt.xticks(fontsize=20)
plt.ylabel("Ground Truth", fontsize=20); plt.yticks(fontsize=20)
plt.show()

plt.figure(figsize = (8,8))
plt.title("Pred / GT", fontsize=20)
sns.heatmap(conf_rate_df, annot=True, cmap='RdYlGn', linewidths=0.1, vmin=0, vmax=1.)
plt.xlabel("Predict", fontsize=20); plt.xticks(fontsize=20)
plt.ylabel("Ground Truth", fontsize=20); plt.yticks(fontsize=20)
plt.show()

## 3-3. Recall, Precision, F-Measure

- 機械学習で一般的な評価指標であるRecall, Precision, F-Measureも求めてみます．
- Confusion Matrixを用いて，True Positive, False Positive, True Negative, False Negativeの各値を，クラスごとに求めます．

In [0]:
col=["TP","FP","TN","FN","Precision","Recall","F"]

df_evaluation = pd.DataFrame(columns=col)

for i in range(conf_matrix.shape[0]):
    tp = conf_matrix[i][i]
    fp = np.sum(conf_matrix[:,i]) - tp
    fn = np.sum(conf_matrix[i]) - tp
    tn = np.sum(conf_matrix) - tp - fp - fn
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    F = 2 * recall * precision / (recall + precision)
    df_evaluation = df_evaluation.append(pd.DataFrame([[tp,fp,tn,fn,precision,recall,F]],
                                                                      columns=col),
                                                                     ignore_index=True)

In [0]:
df_evaluation

- 評価値の統計量を計算します，

In [0]:
df_evaluation.describe()

## 3-4. モデルの保存

- `model.save`で，現在のモデルを保存できます．

In [0]:
model.save("nn_mnist.h5")

- 保存したモデルは以下のように呼び出せます．

In [0]:
# 保存したモデルを読み込む場合は実行
# model = keras.models.load_model('./mnist_data.h5')